In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
import multiprocessing
from torch import autograd
from fastai.conv_learner import *
from fastai.transforms import TfmType
from fasterai.transforms import *
from fasterai.images import *
from fasterai.dataset import *
from fasterai.visualize import *
from fasterai.callbacks import *
from fasterai.loss import *
from fasterai.modules import *
from fasterai.wgan import *
from fasterai.generators import *
from fastai.torch_imports import *
from pathlib import Path
from itertools import repeat
import tensorboardX
torch.cuda.set_device(3)
plt.style.use('dark_background')
torch.backends.cudnn.benchmark=True


In [3]:
#TEST NOTES:  With attention, 2x Gen Filter, just higher learning ratio
IMAGENET = Path('data/imagenet/ILSVRC/Data/CLS-LOC/train')
CIFAR10 = Path('data/cifar10/train')

proj_id = 'bwc_attn2x_long_4'
TENSORBOARD_PATH = Path('data/tensorboard/' + proj_id)

c_lr=5e-4
c_lrs = np.array([c_lr,c_lr,c_lr])

g_lr=c_lr/100
g_lrs = np.array([g_lr/1000,g_lr/10,g_lr])

x_tfms = [BlackAndWhiteTransform()]
torch.backends.cudnn.benchmark=True

## Training

In [4]:
netG = Unet34(nf_factor=2, self_attention=True).cuda()
#netGVis = ModelVisualizationHook(TENSORBOARD_PATH, netG, 'netG')
#load_model(netG, gpath)

netD = DCCritic(ni=3, nf=128, scale=32, self_attention=True).cuda()
#netDVis = ModelVisualizationHook(TENSORBOARD_PATH, netD, 'netD')
#load_model(netD, dpath)

In [5]:
trainer = WGANTrainer(netD=netD, netG=netG, genloss_fns=[FeatureLoss(multiplier=1e5)])
trainerVis = WganVisualizationHook(TENSORBOARD_PATH, trainer, 'trainer', jupyter=False, visual_iters=400)

In [6]:
scheds=[]

scheds.extend(WGANTrainSchedule.generate_schedules(szs=[64,64,64,64,64,64], bss=[64,64,64,64,64,64], path=IMAGENET, x_tfms=x_tfms, keep_pcts=[1.0,1.0,1.0,1.0,1.0,1.0], 
        save_base_name=proj_id, c_lrs=c_lrs, g_lrs=g_lrs, lrs_unfreeze_factor=0.1, gen_freeze_tos=[-1,-1,-1,0,0,0]))

scheds.extend(WGANTrainSchedule.generate_schedules(szs=[128,128,128,128], bss=[16,16,16,16], path=IMAGENET, x_tfms=x_tfms, keep_pcts=[1.0,1.0,1.0,1.0], save_base_name=proj_id, 
        c_lrs=c_lrs/4, g_lrs=g_lrs/4, lrs_unfreeze_factor=0.1, gen_freeze_tos=[-1,-1,0,0]))

scheds.extend(WGANTrainSchedule.generate_schedules(szs=[256,256], bss=[4,4], path=IMAGENET, x_tfms=x_tfms, keep_pcts=[1.0,1.0], save_base_name=proj_id, 
        c_lrs=c_lrs/16, g_lrs=g_lrs/16, lrs_unfreeze_factor=0.1, gen_freeze_tos=[-1,0]))

scheds.extend(WGANTrainSchedule.generate_schedules(szs=[384,384], bss=[1,1], path=IMAGENET, x_tfms=x_tfms, keep_pcts=[1.0,1.0], save_base_name=proj_id, 
        c_lrs=c_lrs/64, g_lrs=g_lrs/64, lrs_unfreeze_factor=0.1, gen_freeze_tos=[-1,0]))

In [ ]:
trainer.train(scheds=scheds)

  3%|▎         | 660/19819 [09:04<4:30:26,  1.18it/s]
WDist -750.4224853515625; RScore 521.6626586914062; FScore -228.7598419189453; GAddlLoss 6198.74951171875; Iters: 39970; GCost: 58.0301513671875; EpsPenalty: 27.213193893432617
  3%|▎         | 680/19819 [09:19<2:17:34,  2.32it/s]
WDist -315.41241455078125; RScore 224.84149169921875; FScore -90.57093048095703; GAddlLoss 6357.22509765625; Iters: 39980; GCost: 352.89520263671875; EpsPenalty: 5.0553693771362305
  4%|▎         | 700/19819 [09:25<1:42:23,  3.11it/s]
WDist -761.8289794921875; RScore 580.4782104492188; FScore -181.35076904296875; GAddlLoss 6108.01220703125; Iters: 39990; GCost: -300.4927673339844; EpsPenalty: 33.695491790771484
  4%|▎         | 720/19819 [09:34<1:43:30,  3.08it/s]
WDist 292.4607238769531; RScore -331.91961669921875; FScore -39.458885192871094; GAddlLoss 5598.3896484375; Iters: 40000; GCost: 38.905948638916016; EpsPenalty: 11.01706314086914
  4%|▎         | 740/19819 [10:49<1:51:14,  2.86it/s]
WDist 281.742